In [1]:
def siglas(nome_time):
    if nome_time == 'Bauru':
        return 'BAU'
    
    elif nome_time == 'MOGI1':
        return 'MOG'
    
    elif nome_time == 'VipTech CMB':
        return 'CMO'
    
    elif nome_time == 'Brasília':
        return 'BSB'
    
    elif nome_time == 'Paulistano':
        return 'CAP'
    
    elif nome_time == 'Flamengo':
        return 'FLA'
    
    elif nome_time == 'Minas':
        return 'MIN'
    
    elif nome_time == 'UNIFACISA':
        return 'UFC'
    
    elif nome_time == 'Cerrado Basquete':
        return 'CER'
    
    elif nome_time == 'SESI Franca':
        return 'FRA'
    
    elif nome_time == 'Corinthians':
        return 'COR'
    
    elif nome_time == 'Pinheiros':
        return 'PIN'
    
    elif nome_time == 'Fortaleza B. C':
        return 'FOR'
    
    elif nome_time == 'KTO Caxias do Sul':
        return 'CAX'
    
    elif nome_time == 'Pato':
        return 'PAT'
    
    elif nome_time == 'São Paulo':
        return 'SPF'

# Bibliotecas

In [2]:
import pandas as pd
import re
from datetime import datetime
import datetime as dt

In [3]:
# arquivo = "tabela_16_Corinthians_x_Paulistano.csv"
# arquivo = "tabela_45_KTO Caxias do Sul_x_Corinthians.csv"
arquivo = "tabela_16_Corinthians_x_Paulistano.csv"
# arquivo = "tabela_51_Flamengo_x_Brasília.csv"
# arquivo = "tabela_10_Bauru_x_Flamengo.csv"

In [4]:
df = pd.read_csv(arquivo)

### Localizar as expressões regulares

In [5]:
expressao_regular = re.findall(r'[A-Z].*?[.]', arquivo)
expressao_regular

['Corinthians_x_Paulistano.']

In [6]:
expressao_regular = str(expressao_regular).strip('[]')
expressao_regular0 = expressao_regular.split('_x_')

# criar um df que identifique o Mogi e colocar mogi das cruzes

In [7]:
casa = expressao_regular0[0]
nome_time_casa = casa.replace("'", "")
nome_time_casa

'Corinthians'

In [8]:
fora = expressao_regular0[1]
nome_time_fora = fora.replace(".'", "")
nome_time_fora

'Paulistano'

### Informações básicas sobre o jogo para que completar a tabela

In [9]:
data_hoje = datetime.today().strftime('%d/%m/%Y')
dia_do_jogo = '02/01/2021'
temporada = 2019
sigla_time_a = siglas(nome_time_casa)
sigla_time_b = siglas(nome_time_fora)
casa = 'casa'
fora = 'fora'
classificatoria = '1 Turno'

### Local dos arquivos para salvar

In [10]:
arquivo = 'RELATORIO'
arquivo1 = f'informacoes_dos_times/{nome_time_casa}'
arquivo2 = f'informacoes_dos_times/{nome_time_fora}'
img_casa = f'{arquivo1}/{nome_time_casa}'
img_fora = f'{arquivo2}/{nome_time_fora}'

In [11]:
informações_jogo =  pd.DataFrame({
    'Nomes':         [nome_time_casa, nome_time_fora],
    'Siglas':        [sigla_time_a, sigla_time_b],
    'Mandante':      ['casa', 'fora'],
    'Classificação': [classificatoria, classificatoria],
    'Temporada':     [temporada, temporada],
    'Dia':           [dia_do_jogo, dia_do_jogo],
    'Arquivo':       [arquivo1, arquivo2],
    'Imagem':        [img_casa, img_fora]
})
informações_jogo

,Nomes,Siglas,Mandante,Classificação,Temporada,Dia,Arquivo,Imagem
0,Corinthians,COR,casa,1 Turno,2019,02/01/2021,informacoes_dos_times/Corinthians,informacoes_dos_times/Corinthians/Corinthians
1,Paulistano,CAP,fora,1 Turno,2019,02/01/2021,informacoes_dos_times/Paulistano,informacoes_dos_times/Paulistano/Paulistano


In [12]:
informações_jogo.to_csv('informacao_jogo.csv')

# Limpeza dos dados

In [13]:
# primeiro evitar NAN (acredite!!! tem isso no site)
df.dropna(subset=['Tempo'], inplace=True)

* mudança do tempo
* colocar todos em segundos para facilitar a vida
* tem jogos que o site apresenta numeros inteiros (1, 12, 134, 1000)
* para isso localizei o valores errados e concertei

In [14]:
mudar_hora = []
for x in df['Tempo']:
    if re.findall(r'..:..', x):
        mudar_hora.append(x)
    else:
        if re.findall(r'....', x):
            x = x[0:2] + ':' + x[2:4]
            mudar_hora.append(x)
        elif re.findall(r'...', x):
            x = '0' + x[0] + ':' + x[1:3]
            mudar_hora.append(x)
        elif re.findall(r'..', x):
            x = '00:' + x
            mudar_hora.append(x)
        elif re.findall(r'.', x):
            x = '00:0' + x
            mudar_hora.append(x)

df['Tempo_2'] = mudar_hora
df.drop('Tempo', axis=1, inplace=True)

In [15]:
df['Tempo_2'] = df['Tempo_2'].apply(lambda x: dt.datetime.strptime(x, '%M:%S'))

In [16]:
df['Tempo_2'] = df['Tempo_2'].apply(lambda x: dt.time(x.hour, x.minute, x.second))

In [17]:
df['Tempo_2'] = df['Tempo_2'].apply(lambda x: (x.hour * 60 + x.minute) * 60 + x.second)

In [18]:
# transforma os dados para números inteiros
df['Quarto'] = df['Quarto'].apply(lambda l: int(l))

In [19]:
# modificar o tempo decrescente para crescente (* -1)
# acrescentar o tempo de cada quarto (primeiro quarto termina em 600s, o segundo quarto 2*600 = 1200 ...)
tempo_novo = []
for x, y in zip(df['Quarto'], df['Tempo_2']):
    if x == 1:
        a = (y - (600 * 1)) * -1
        tempo_novo.append(a)
    elif x == 2:
        a = (y - (600 * 2)) * -1
        tempo_novo.append(a)
    elif x == 3:
        a = (y - (600 * 3)) * -1
        tempo_novo.append(a)
    elif x == 4:
        a = (y - (600 * 4)) * -1
        tempo_novo.append(a)
    elif x == 5:
        a = (y - (600 * 4.5)) * -1
        tempo_novo.append(a)
    elif x == 6:
        a = (y - (600 * 5)) * -1
        tempo_novo.append(a)
    elif x == 7:
        a = (y - (600 * 5.5)) * -1
        tempo_novo.append(a)

df['Tempo'] = tempo_novo
df.drop('Tempo_2', axis=1, inplace=True)

In [20]:
# deixando o DataFrame nessa ordem de colunas
df = df[['Quarto', 'Tempo', 'placar_casa', 'placar_visitante', 'Time', 'Indicador', 'Nome']]

In [21]:
# Acrescentado colunas
# diferenca_placar_casa
# diferenca_placar_visitante
df['diferenca_placar_casa'] = df['placar_casa'] - df['placar_visitante']
df['diferenca_placar_visitante'] = df['placar_visitante'] - df['placar_casa']
df["diferenca_placar_absoluto"] = df.loc[:, ("diferenca_placar_casa")].abs()
df

,Quarto,Tempo,placar_casa,placar_visitante,Time,Indicador,Nome,diferenca_placar_casa,diferenca_placar_visitante,diferenca_placar_absoluto
0,6,3000.0,98,100,NaN,fim_partida,NaN,-2,2,2
1,6,3000.0,98,100,NaN,fim_quarto,NaN,-2,2,2
2,6,2996.0,98,100,COR,AS,Fischer,-2,2,2
3,6,2996.0,98,100,COR,3_Pts_C,Fuller,-2,2,2
4,6,2994.0,95,100,COR,RO,Renato,-5,5,5
...,...,...,...,...,...,...,...,...,...,...
647,1,0.0,0,0,COR,substituicao_entra,Miller,0,0,0
648,1,0.0,0,0,COR,substituicao_entra,L Cauê,0,0,0
649,1,0.0,0,0,COR,substituicao_entra,Fuller,0,0,0
650,1,0.0,0,0,COR,substituicao_entra,Fischer,0,0,0


In [22]:
df.to_csv('df.csv')